# 　　　　　　　　　　　<    2019 공공데이터 공모전   >

　
　
　
　

# 공원일몰제에 대비한 미세먼지, 취약 계층을 고려한 미집행 공원 우선 순위 선정
　
 　
  　
   

## 　　　　　　　　　　　　　　　　　　　　　　　　　　　　The freedom of Data (데자유)

#### 　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　* 팀장:　편신범　 * 팀원:  강정호,  국승지,  이선민

In [ ]:
목차

0. 목표 데이터 확인

1. 서울시 공원
  1.1 전국공원표준데이터 - 서울
    1.1.1 행정구 뽑아내기
    1.1.2 사용하지 않는 컬럼 삭제

  1.2 마포구가 누락되어 마포구 추가
    1.2.1 행정구 뽑아내기
    1.2.2 마포구 사용하지 않는 컬럼 삭제

  1.3 서울시 공원 전체 df생성
    
  1.4 필요한 컬럼만 골라내기

  1.5 그룹 나누기 groupby
    1.5.1 구코드 기준으로 그룹
    1.5.2 현재 구별 공원면적
    1.5.3 현재 구별 공원 수
    1.5.4 현재 구별 공원면적 + 현재 구별 공원 수
    
2. 미집행 면적 현황
  2.1 필요한 컬럼만 선택하기
    @이슈 발생 & 해결
  2.2 공원현황df과 미집행면적현황df 합치기
  2.3 파생변수 생성 - ['예상총면적'] = ['공원면적'] + ['미집행면적']
  2.4 전체 공원df 확인하기

3. 생활인구
  3.1 생활인구.csv 불러오기
  3.2 공원df + 생활인구df 합치기
  3.3 파생변수 만들기
    
4. 시간대별 생활인구는 이 분석에서 큰 의미가 없다
  각 구, 하루별 평균값으로 분석진행

5. 미세먼지 데이터
  5.1 미세먼지 불러오기
  5.2 공원df + 생활인구df + 미세먼지df 합치기
  5.3 NO2 컬럼 삭제
    
6. 2017, 2018 일별 기상관측
  6.1 일별 기상관측 2017
  6.2 일별 기상관측 2018
  6.3 2017과 2018 append해서 하나로 만듦
  6.4 지역 이름이 '중구구' 라서 '중구'로 바꿔줌
  6.5 온도습도df + 기존 df
    
7. 지하철 데이터
  7.1 지하철자료 불러오기
  7.2 지하철df + 기존df

8. 각 구청 위도 경도 추가

*각 지역별 데이터로 변환 

9. 도시농업공원 이슈 발생
  9.1 인덱스를 '지역'으로 변경
  9.2 노원구의 도시농업공원 2곳의 면적 제외
  9.3 양천구의 도시농업공원 1곳 면적 제외
    
10. 취약계층 데이터
  10.1 취약계층 불러오기
  10.2 기존df + 취약계층비율df
  10.3 불투수면적 & 구 전체면적df + 기존df
    10.3.1 서울시 지역df 만들기
    10.3.2 구 전체면적 df만들기
    10.3.3 불투수면적 df만들기
    10.3.4 구전체면적df + 불투수면적df

*1인당 면적 수정

* 최종 dataframe 생성 & 확인

11. final rank 3가지 기준에 부합하는 지역 찾기
  11.1 미세먼지 높은구
  11.2 공원면적비율 적은 수 
    11.2.1 파생변수 - 공원면적비율 생성
  11.3 흡수율 적은 수(분석 제외)
  11.4 취약계층
    11.4.1 취약계층 비율(분석 제외)
    11.4.2 취약계층 수
  11.5 공원1m2당 인구수 (분석 제외)
  11.6 1인당 공원면적
  11.7 4개 df 합치기
  11.8 최종 순위

In [824]:
import pandas as pd

# 1. 서울시 공원

## 1.1 전국공원표준데이터 - 서울

2018,2019년 업데이트된 데이터

In [825]:
park_all = pd.read_csv('seoul_park_all.csv', encoding='cp949')

In [826]:
park_all.head()

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명,Unnamed: 19
0,11110-00030,창신공원,어린이공원,서울특별시 종로구 창신 23-362,서울특별시 종로구 창신 23-362,37.578276,127.012931,710.0,NaN,"시소,조합놀이",NaN,NaN,NaN,2013-10-10,서울특별시 종로구청,02-2148-4156,2018-08-16,3000000,서울특별시 종로구,NaN
1,11110-00031,당고개공원,어린이공원,서울특별시 종로구 창신 23-350,서울특별시 종로구 창신 23-350,37.575762,127.012382,860.0,NaN,"그네,조합놀이",NaN,NaN,NaN,2013-10-10,서울특별시 종로구청,02-2148-4156,2018-08-16,3000000,서울특별시 종로구,NaN
2,11110-00032,무악공원,어린이공원,서울특별시 종로구 무악동 산2-83,서울특별시 종로구 무악동 산2-83,37.577525,126.960547,5136.0,NaN,NaN,음수대,NaN,NaN,2015-09-13,서울특별시 종로구청,02-2148-4156,2018-08-16,3000000,서울특별시 종로구,NaN
3,11110-00033,창덕공원,근린공원,서울특별시 종로구 율곡로 99 (와룡동),서울특별시 종로구 와룡동 2-71,37.580274,126.991785,491328.0,NaN,NaN,음수대,NaN,NaN,NaN,서울특별시 종로구청,NaN,2018-08-16,3000000,서울특별시 종로구,NaN
4,11110-00034,청진공원,근린공원,서울특별시 종로구 종로3길 24-29 (청진동),서울특별시 종로구 청진동 177,37.571796,126.979991,3341.0,NaN,NaN,NaN,NaN,NaN,NaN,서울특별시 종로구청,NaN,2018-08-16,3000000,서울특별시 종로구,NaN


In [827]:
park_all.columns

Index(['관리번호', '공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자', '제공기관코드', '제공기관명',
       'Unnamed: 19'],
      dtype='object')

## 1.1.1 행정구 뽑아내기

#### ['제공기관명'] 에서 '구' 정보 확인가능

In [828]:
park_all['제공기관명']

0       서울특별시 종로구
1       서울특별시 종로구
2       서울특별시 종로구
3       서울특별시 종로구
4       서울특별시 종로구
          ...    
1624    서울특별시 동작구
1625    서울특별시 동작구
1626    서울특별시 동작구
1627    서울특별시 동작구
1628    서울특별시 동작구
Name: 제공기관명, Length: 1629, dtype: object

#### 서울시 25개 구에서 1개구가 누락됨을 확인함.

#### 누락된 구는 '마포구'

In [829]:
len(park_all['제공기관명'].value_counts())

24

In [830]:
park_all['구코드'] = park_all['제공기관명'].apply(lambda x: x.split(' ')[1])
park_all.head()

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),...,공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,제공기관코드,제공기관명,Unnamed: 19,구코드
0,11110-00030,창신공원,어린이공원,서울특별시 종로구 창신 23-362,서울특별시 종로구 창신 23-362,37.578276,127.012931,710.0,NaN,"시소,조합놀이",...,NaN,NaN,2013-10-10,서울특별시 종로구청,02-2148-4156,2018-08-16,3000000,서울특별시 종로구,NaN,종로구
1,11110-00031,당고개공원,어린이공원,서울특별시 종로구 창신 23-350,서울특별시 종로구 창신 23-350,37.575762,127.012382,860.0,NaN,"그네,조합놀이",...,NaN,NaN,2013-10-10,서울특별시 종로구청,02-2148-4156,2018-08-16,3000000,서울특별시 종로구,NaN,종로구
2,11110-00032,무악공원,어린이공원,서울특별시 종로구 무악동 산2-83,서울특별시 종로구 무악동 산2-83,37.577525,126.960547,5136.0,NaN,NaN,...,NaN,NaN,2015-09-13,서울특별시 종로구청,02-2148-4156,2018-08-16,3000000,서울특별시 종로구,NaN,종로구
3,11110-00033,창덕공원,근린공원,서울특별시 종로구 율곡로 99 (와룡동),서울특별시 종로구 와룡동 2-71,37.580274,126.991785,491328.0,NaN,NaN,...,NaN,NaN,NaN,서울특별시 종로구청,NaN,2018-08-16,3000000,서울특별시 종로구,NaN,종로구
4,11110-00034,청진공원,근린공원,서울특별시 종로구 종로3길 24-29 (청진동),서울특별시 종로구 청진동 177,37.571796,126.979991,3341.0,NaN,NaN,...,NaN,NaN,NaN,서울특별시 종로구청,NaN,2018-08-16,3000000,서울특별시 종로구,NaN,종로구


## 1.1.2 사용하지 않는 컬럼 삭제

In [831]:
park_all = park_all.drop(['제공기관코드', 'Unnamed: 19', '제공기관명'], axis = 1)

## 1.2 마포구가 누락되어 마포구 추가

2019년 데이터

https://www.data.go.kr/dataset/15032365/fileData.do

In [832]:
park_mapo = pd.read_csv('서울특별시_마포구_도시공원정보_20190722.csv', encoding='cp949')
park_mapo.head()

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자
0,11440-00001,상암공원,근린공원,NaN,서울특별시 마포구 상암동 산26-39,37.578874,126.883114,161153,NaN,NaN,NaN,NaN,NaN,2000-09-27,서울특별시 마포구,02-3153-9553,2019-07-22
1,11440-00002,매봉산공원,근린공원,NaN,서울특별시 마포구 상암동 1631,37.572886,126.892089,13137,NaN,NaN,NaN,NaN,NaN,2006-11-16,서울특별시 마포구,02-3153-9553,2019-07-22
2,11440-00003,구룡공원,근린공원,NaN,서울특별시 마포구 상암동 1583,37.582790,126.883715,14287,NaN,NaN,NaN,NaN,NaN,2000-09-27,서울특별시 마포구,02-3153-9553,2019-07-22
3,11440-00004,와우공원,근린공원,NaN,서울특별시 마포구 창전동3-231 등 59필지,27.551606,126.929047,73590,NaN,NaN,NaN,NaN,NaN,1940-03-12,서울특별시 마포구,02-3153-9553,2019-07-22
4,11440-00005,부엉이공원,근린공원,NaN,서울특별시 마포구 상암동1735,37.584646,126.883282,17640,NaN,NaN,NaN,NaN,NaN,2006-09-07,서울특별시 마포구,02-3153-9553,2019-07-22


In [833]:
park_mapo.columns

Index(['관리번호', '공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자'],
      dtype='object')

## 1.2.1  행정구 뽑아내기

#### ['관리기관명'] 에서 '구' 정보 확인가능

In [834]:
park_mapo['관리기관명']

0     서울특별시 마포구
1     서울특별시 마포구
2     서울특별시 마포구
3     서울특별시 마포구
4     서울특별시 마포구
        ...    
76    서울특별시 마포구
77    서울특별시 마포구
78    서울특별시 마포구
79    서울특별시 마포구
80    서울특별시 마포구
Name: 관리기관명, Length: 81, dtype: object

In [835]:
park_mapo['구코드'] = park_mapo['관리기관명'].apply(lambda x: x.split(' ')[1])
park_mapo.head()

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,구코드
0,11440-00001,상암공원,근린공원,NaN,서울특별시 마포구 상암동 산26-39,37.578874,126.883114,161153,NaN,NaN,NaN,NaN,NaN,2000-09-27,서울특별시 마포구,02-3153-9553,2019-07-22,마포구
1,11440-00002,매봉산공원,근린공원,NaN,서울특별시 마포구 상암동 1631,37.572886,126.892089,13137,NaN,NaN,NaN,NaN,NaN,2006-11-16,서울특별시 마포구,02-3153-9553,2019-07-22,마포구
2,11440-00003,구룡공원,근린공원,NaN,서울특별시 마포구 상암동 1583,37.582790,126.883715,14287,NaN,NaN,NaN,NaN,NaN,2000-09-27,서울특별시 마포구,02-3153-9553,2019-07-22,마포구
3,11440-00004,와우공원,근린공원,NaN,서울특별시 마포구 창전동3-231 등 59필지,27.551606,126.929047,73590,NaN,NaN,NaN,NaN,NaN,1940-03-12,서울특별시 마포구,02-3153-9553,2019-07-22,마포구
4,11440-00005,부엉이공원,근린공원,NaN,서울특별시 마포구 상암동1735,37.584646,126.883282,17640,NaN,NaN,NaN,NaN,NaN,2006-09-07,서울특별시 마포구,02-3153-9553,2019-07-22,마포구


## 1.2.2 마포구 사용하지 않는 컬럼 삭제

서울시전체df와 마포구df의 컬럼이 동일함을 확인함

In [836]:
park_mapo.columns

Index(['관리번호', '공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자', '구코드'],
      dtype='object')

In [837]:
park_all.columns

Index(['관리번호', '공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자', '구코드'],
      dtype='object')

In [838]:
park_all.columns == park_mapo.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

## 1.3 서울시 공원 전체 df생성

In [839]:
park_all = park_all.append(park_mapo).reset_index(drop=True)
park_all.head()

,관리번호,공원명,공원구분,소재지도로명주소,소재지지번주소,위도,경도,공원면적,공원보유시설(운동시설),공원보유시설(유희시설),공원보유시설(편익시설),공원보유시설(교양시설),공원보유시설(기타시설),지정고시일,관리기관명,전화번호,데이터기준일자,구코드
0,11110-00030,창신공원,어린이공원,서울특별시 종로구 창신 23-362,서울특별시 종로구 창신 23-362,37.578276,127.012931,710.0,NaN,"시소,조합놀이",NaN,NaN,NaN,2013-10-10,서울특별시 종로구청,02-2148-4156,2018-08-16,종로구
1,11110-00031,당고개공원,어린이공원,서울특별시 종로구 창신 23-350,서울특별시 종로구 창신 23-350,37.575762,127.012382,860.0,NaN,"그네,조합놀이",NaN,NaN,NaN,2013-10-10,서울특별시 종로구청,02-2148-4156,2018-08-16,종로구
2,11110-00032,무악공원,어린이공원,서울특별시 종로구 무악동 산2-83,서울특별시 종로구 무악동 산2-83,37.577525,126.960547,5136.0,NaN,NaN,음수대,NaN,NaN,2015-09-13,서울특별시 종로구청,02-2148-4156,2018-08-16,종로구
3,11110-00033,창덕공원,근린공원,서울특별시 종로구 율곡로 99 (와룡동),서울특별시 종로구 와룡동 2-71,37.580274,126.991785,491328.0,NaN,NaN,음수대,NaN,NaN,NaN,서울특별시 종로구청,NaN,2018-08-16,종로구
4,11110-00034,청진공원,근린공원,서울특별시 종로구 종로3길 24-29 (청진동),서울특별시 종로구 청진동 177,37.571796,126.979991,3341.0,NaN,NaN,NaN,NaN,NaN,NaN,서울특별시 종로구청,NaN,2018-08-16,종로구


## 1.4 필요한 컬럼만 골라내기

In [840]:
park_all.columns

Index(['관리번호', '공원명', '공원구분', '소재지도로명주소', '소재지지번주소', '위도', '경도', '공원면적',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자', '구코드'],
      dtype='object')

In [841]:
park_all = park_all.drop(['관리번호', '소재지도로명주소', '소재지지번주소', '위도', '경도',
       '공원보유시설(운동시설)', '공원보유시설(유희시설)', '공원보유시설(편익시설)', '공원보유시설(교양시설)',
       '공원보유시설(기타시설)', '지정고시일', '관리기관명', '전화번호', '데이터기준일자'], axis = 1)

In [842]:
park_all.head()

,공원명,공원구분,공원면적,구코드
0,창신공원,어린이공원,710.0,종로구
1,당고개공원,어린이공원,860.0,종로구
2,무악공원,어린이공원,5136.0,종로구
3,창덕공원,근린공원,491328.0,종로구
4,청진공원,근린공원,3341.0,종로구


## 1.5 그룹 나누기 groupby

### 1.5.1 구코드 기준으로 그룹

In [843]:
park_gu_group = park_all.groupby(park_all.구코드)
park_gu_group.groups

{'강남구': Int64Index([779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
             ...
             903, 904, 905, 906, 907, 908, 909, 910, 911, 912],
            dtype='int64', length=134),
 '강동구': Int64Index([1050, 1051, 1052, 1053, 1054, 1055, 1056, 1057, 1058, 1059, 1060,
             1061, 1062, 1063, 1064, 1065, 1066, 1067, 1068, 1069, 1070, 1071,
             1072, 1073, 1074, 1075, 1076, 1077, 1078, 1079, 1080, 1081, 1082,
             1083, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1091, 1092, 1093,
             1094, 1095, 1096, 1097, 1098, 1099, 1100, 1101, 1102, 1103, 1104,
             1105, 1106, 1107, 1108, 1109, 1110, 1111, 1112, 1113, 1114, 1115,
             1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123],
            dtype='int64'),
 '강북구': Int64Index([1486, 1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496,
             1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507,
             1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 

### 1.5.2 현재 구별 공원면적 

In [844]:
park_m2_now = park_gu_group.sum().reset_index()
park_m2_now.head()

,구코드,공원면적
0,강남구,2146595.1
1,강동구,1934992.9
2,강북구,1567125.5
3,강서구,3209226.5
4,관악구,442644.0


### 1.5.3 현재 구별 공원 수

In [845]:
park_count = park_gu_group.count().reset_index()
park_count = park_count.drop(['공원면적','공원구분'], axis = 1)
park_count = park_count.rename(columns = {'공원명' : '공원수'})


# 컬럼명 변경  참조https://datasheet.tistory.com/8

park_count.head()

,구코드,공원수
0,강남구,134
1,강동구,74
2,강북구,45
3,강서구,139
4,관악구,76


### 1.5.4 현재 구별 공원면적 + 현재 구별 공원 수

In [846]:
park_now = park_m2_now.merge(park_count, on='구코드', how = 'inner')
park_now.head()

,구코드,공원면적,공원수
0,강남구,2146595.1,134
1,강동구,1934992.9,74
2,강북구,1567125.5,45
3,강서구,3209226.5,139
4,관악구,442644.0,76


# 2. 미집행 면적 현황 


## 2018년 자료만 사용함 
http://kosis.kr/statisticsList/statisticsListIndex.do?menuId=M_01_01&vwcd=MT_ZTITLE&parmTabId=M_01_01&parentId=H.1;H1.2;315_31502.3;#SelectStatsBoxDiv

In [847]:
park_me = pd.read_csv('공원_미집행현황_시군구__20190816173446.csv', encoding='utf-8')

In [848]:
park_me.head()

,소재지(시군구)별(1),소재지(시군구)별(2),2017,2017.1,2017.2,2017.3,2017.4,2018,2018.1,2018.2,2018.3,2018.4
0,소재지(시군구)별(1),소재지(시군구)별(2),(부분)미집행결정면적 (㎡),집행면적 (㎡),집행비율 (%),미집행면적 (㎡),추정사업비계 (백만원),(부분)미집행결정면적 (㎡),집행면적 (㎡),집행비율 (%),미집행면적 (㎡),추정사업비계 (백만원)
1,서울특별시,서울특별시,88040753,47100949,53.5,40939804,7477299,88040753,47165101,53.57,40875652,7455026
2,서울특별시,중구,205,201,98.05,4,0,205,201,98.05,4,0
3,서울특별시,용산구,186246,8660,4.65,177586,508723,169296,8660,5.12,160636,251618
4,서울특별시,성동구,193742,53292,27.51,140450,4500,193742,53292,27.51,140450,4500


## 2.1 필요한 컬럼만 선택하기

In [849]:
park_me = park_me.drop(['2017', '2017.1', '2017.2', '2017.3','2017.4'], axis = 1)
park_me = park_me.drop(index=[0,1]).reset_index(drop = True)
park_me = park_me.rename(columns = {'소재지(시군구)별(2)': '구코드','2018.3' : '미집행면적','2018.4' : '추정사업비계 (백만원)'})
park_me = park_me.drop(['소재지(시군구)별(1)', '2018','2018.1','2018.2'], axis = 1)

### @이슈 발생: 미집행 면적 현황df에서 4개 구가 누락됨을 확인함 (금천 송파 양천 종로)

In [850]:
len(park_me)

21

### @이슈 발생 해결방법
구 전체면적 중 미집행면적의 비율 계산하고

모든 구의 비율 평균을 누락된 구의 전체면적에 곱해서 미집행면적 임의로 산출하기?

In [851]:
gu_map = """39703000
25236000
23819000
42077000
29969000
17874000
20096000
12946000
35755000
20777000
14588000
16457000
23479000
17813000
46956000
16422000
24877000
33109000
17346000
24639000
21968000
31333000
23128000
9980000
18314000"""


In [852]:
gu_map_list = gu_map.split('\n')
gu_map_df = pd.DataFrame(gu_map_list).rename(columns = {0:'구 전체면적'}).reset_index()
gu_map_df

,index,구 전체면적
0,0,39703000
1,1,25236000
2,2,23819000
3,3,42077000
4,4,29969000
5,5,17874000
6,6,20096000
7,7,12946000
8,8,35755000
9,9,20777000


In [853]:
gu_str = """강남구 강동구 강북구 강서구 관악구 광진구 구로구 금천구 노원구 도봉구 동대문구 동작구 마포구 서대문구 서초구 성동구 성북구 송파구 양천구 영등포구 용산구 은평구 종로구 중구 중랑구"""
gu_list = gu_str.split()
gu_list_df = pd.DataFrame(gu_list).rename(columns ={0:'구코드'}).reset_index()
gu_list_df.head()

,index,구코드
0,0,강남구
1,1,강동구
2,2,강북구
3,3,강서구
4,4,관악구


In [854]:
seoul_m2 = gu_list_df.merge(gu_map_df, on='index', how='left').drop('index', axis = 1)
seoul_m2 = seoul_m2.merge(park_me, on='구코드', how='left')
seoul_m2['미집행면적'] = seoul_m2['미집행면적'].astype('float')
seoul_m2['구 전체면적'] = seoul_m2['구 전체면적'].astype('float')
seoul_m2['미집행비율'] = seoul_m2['미집행면적'] / seoul_m2['구 전체면적']
yet_ratio = seoul_m2['미집행비율'].mean()

# mean 하면 nan행은 자동으로 빠지는 듯

In [855]:
yet_ratio

0.03235634154677126

In [856]:
seoul_m2 = seoul_m2.fillna('nan')
for idx, val in enumerate(seoul_m2['미집행면적']): 
    if val == 'nan':
        seoul_m2['미집행면적'][idx] = yet_ratio * seoul_m2['구 전체면적'][idx] 
seoul_m2 = seoul_m2.drop(['추정사업비계 (백만원)','미집행비율'], axis = 1)

C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [857]:
seoul_m2

,구코드,구 전체면적,미집행면적
0,강남구,39703000.0,805207
1,강동구,25236000.0,898745
2,강북구,23819000.0,468932
3,강서구,42077000.0,1.47243e+06
4,관악구,29969000.0,289624
5,광진구,17874000.0,12634
6,구로구,20096000.0,313974
7,금천구,12946000.0,418885
8,노원구,35755000.0,852658
9,도봉구,20777000.0,820064


In [858]:
park_me = park_me.merge(seoul_m2, on=['구코드'], how = 'right').drop(['미집행면적_x'], axis = 1).rename(columns = {'미집행면적_y': '미집행면적'})

## 2.2 공원현황df과 미집행면적현황df 합치기

In [859]:
park_total = park_now.merge(park_me, on='구코드', how='outer')

In [860]:
park_total['공원면적'] = park_total['공원면적'].astype('float')
park_total['미집행면적'] = park_total['미집행면적'].astype('float')

## 2.3 파생변수 생성 - ['예상총면적'] = ['공원면적'] + ['미집행면적']

In [861]:
park_total['예상총면적'] = park_total['공원면적'] + park_total['미집행면적']

## 2.4 전체 공원df 확인하기

In [862]:
park_total.head()

,구코드,공원면적,공원수,추정사업비계 (백만원),구 전체면적,미집행면적,예상총면적
0,강남구,2146595.1,134,101053,39703000.0,805207.0,2951802.1
1,강동구,1934992.9,74,37994,25236000.0,898745.0,2833737.9
2,강북구,1567125.5,45,0,23819000.0,468932.0,2036057.5
3,강서구,3209226.5,139,203114,42077000.0,1472431.0,4681657.5
4,관악구,442644.0,76,60150,29969000.0,289624.0,732268.0


### csv로 저장하기

In [863]:
# park_total.to_csv('서울 모든 공원.csv', encoding='cp949', index=0)

# 3. 생활인구

## 2017, 2018년 자료 사용함

# @@@@ 반장님의 flow_gu_final 구글드라이브업로드 자료에 대한 코드 필요함

## 3.1 생활인구.csv 불러오기

In [864]:
flow_gu = pd.read_csv('flow_gu_final_190816.csv', encoding='utf-8', index_col=0).reset_index()

In [865]:
flow_gu['시간대구분'] = flow_gu['시간대구분'].astype('str')
flow_gu.head()

,기준일ID,시간대구분,자치구코드,총생활인구수
0,20170101,0,종로구,283391.7398
1,20170101,0,중구,189272.8069
2,20170101,0,용산구,288680.0998
3,20170101,0,성동구,313852.9667
4,20170101,0,광진구,382976.1003


#### 공원df 구코드 컬럼명을 '자치구코드'로 변경

In [866]:
park_total = park_total.rename(columns = {'구코드':'자치구코드'})

## 3.2 공원df + 생활인구df 합치기

In [867]:
flow_park = flow_gu.merge(park_total, on='자치구코드', how='inner')
flow_park.head()

,기준일ID,시간대구분,자치구코드,총생활인구수,공원면적,공원수,추정사업비계 (백만원),구 전체면적,미집행면적,예상총면적
0,20170101,0,종로구,283391.7398,1491065.2,34,NaN,23128000.0,748337.467294,2.239403e+06
1,20170101,1,종로구,240461.7883,1491065.2,34,NaN,23128000.0,748337.467294,2.239403e+06
2,20170101,2,종로구,231657.5476,1491065.2,34,NaN,23128000.0,748337.467294,2.239403e+06
3,20170101,3,종로구,227495.4741,1491065.2,34,NaN,23128000.0,748337.467294,2.239403e+06
4,20170101,4,종로구,226370.6112,1491065.2,34,NaN,23128000.0,748337.467294,2.239403e+06


## 3.3 파생변수 만들기 

#### 1인당면적 = 공원면적 / 총생활인구수 

#### 공원당인구수 = 총생활인구수 / 공원수

In [868]:
flow_park['1인당면적'] = flow_park['공원면적'] / flow_park['총생활인구수']

In [869]:
flow_park['공원당인구수'] = flow_park['총생활인구수'] / flow_park['공원수']

In [870]:
flow_park.head()

,기준일ID,시간대구분,자치구코드,총생활인구수,공원면적,공원수,추정사업비계 (백만원),구 전체면적,미집행면적,예상총면적,1인당면적,공원당인구수
0,20170101,0,종로구,283391.7398,1491065.2,34,NaN,23128000.0,748337.467294,2.239403e+06,5.261498,8335.051171
1,20170101,1,종로구,240461.7883,1491065.2,34,NaN,23128000.0,748337.467294,2.239403e+06,6.200841,7072.405538
2,20170101,2,종로구,231657.5476,1491065.2,34,NaN,23128000.0,748337.467294,2.239403e+06,6.436506,6813.457282
3,20170101,3,종로구,227495.4741,1491065.2,34,NaN,23128000.0,748337.467294,2.239403e+06,6.554263,6691.043356
4,20170101,4,종로구,226370.6112,1491065.2,34,NaN,23128000.0,748337.467294,2.239403e+06,6.586832,6657.959153


# 4. 시간대별 생활인구는 이 분석에서 큰 의미가 없다
#### 자료데이터양이 너무 많아서 보기힘들다 

In [871]:
test = flow_park.copy()

## 날짜, 구별 groupby

In [872]:
test_group = test.groupby(['기준일ID','자치구코드'])

## 각 구, 하루별 평균값으로 계산

1. 총생활인구, 1인당 면적, 공원당 인구수가 하루 평균이 되어버림.

2. 모든 구 25개 X 730일(2년) = 18,250 행

In [873]:
send = round(test_group.mean(),0).reset_index()
send = send.rename(columns = {'기준일ID':'date', '자치구코드':'지역'})
send['지역'].value_counts()
send['date'] = send['date'].astype('str')
send['date'] = pd.to_datetime(send['date'])
send.head()

,date,지역,총생활인구수,공원면적,공원수,구 전체면적,미집행면적,예상총면적,1인당면적,공원당인구수
0,2017-01-01,강남구,668041.0,2146595.0,134,39703000.0,805207.0,2951802.0,3.0,4985.0
1,2017-01-01,강동구,505314.0,1934993.0,74,25236000.0,898745.0,2833738.0,4.0,6829.0
2,2017-01-01,강북구,332492.0,1567126.0,45,23819000.0,468932.0,2036058.0,5.0,7389.0
3,2017-01-01,강서구,563938.0,3209226.0,139,42077000.0,1472431.0,4681657.0,6.0,4057.0
4,2017-01-01,관악구,529131.0,442644.0,76,29969000.0,289624.0,732268.0,1.0,6962.0


# 5. 미세먼지 데이터

2017, 2018년 데이터

## 5.1 미세먼지 불러오기



## 2017, 2018 미세먼지 데이터 전처리
미세먼지 데이터는 두 개의 폴더로 나누어져 있습니다.
각각에 폴더에는 분기별로 csv 파일이 저장되어 있습니다. (한 폴더당 4개씩 총 8개)
아래의 함수는, 각각의 폴더에 접근하여 미세먼지 데이터들을 전처리 하는 코드입니다.
전처리가 끝난 코드는 """미세먼지 전처리""" 라는 이름을 가진 폴더로 이동하게 됩니다.
코드를 수정해야한다면, 데이터들이 들어있는 data_dir를 수정하면 작동합니다.

In [874]:
# import moduel
import pandas as pd # pandas 모듈
import numpy as np # numpy 모듈
import os # 폴더를 생성하고 다루기 위함
from os import listdir # 폴더 리스트를 불러오기 위함
import re # 정규표현식 사용

def DataPreprocessing(dir_name): # 폴더이름을 인자로 받는 데이터 전처리 함수
    # 데이터가 들어있는 폴더경로
    # '2017' 데이터 폴더에는 1분기, 2분기, 3분기, 4분기의 미세먼지 데이터가 들어있다.
    data_dir = "C:\\Users\\seonmin\\NLP_Hause\\공공데이터 공모전\\test\\{}".format(dir_name)
    # data_dir로 주피터가 작업하는 폴더를 인식하도록 경로 변경
    working_dir = os.chdir(data_dir) 

    # 전처리된 데이터를 내보낼 폴더
    out_dir = "C:\\Users\\seonmin\\NLP_Hause\\공공데이터 공모전\\test\\{}".format('미세먼지_전처리') 
    if not os.path.exists(out_dir): # 만약 경로가 없으면, 
            os.makedirs(out_dir) # 새로운 폴더를 생성하겠습니다.

    # 전처리할 파일들이 들어있는 폴더의 파일 목록을 list형태로 추출
    file_list = os.listdir(data_dir)
    # 최종적으로 합쳐질 빈 데이터 프레임 생성
    Merged_DF = pd.DataFrame(columns=['측정소코드', '측정소명', '측정일시', 
                                      'SO2', 'CO', 'O3', 'NO2', 'PM10', 'PM25'])

    # for문을 돌면서 하나씩 파일을 꺼내옴
    for file in file_list:
        raw_df = pd.read_csv(file, encoding='cp949')
        raw_df = raw_df.rename(columns={'Unnamed: 0': '지역'})
        raw_df = raw_df.loc[:, ~raw_df.columns.str.contains('^Unnamed')] # 정규표현식 활용해서 Unnamed columns 삭제

        # 분석에 사용하지 않을 ['주소'] columns drop
        raw_df = raw_df.drop(['주소'], axis='columns')

        # 데이터 "결측치" 처리 code -> Not a Number로 변경
        raw_df = raw_df.replace([-999,-9999],[np.nan, np.nan])

        # pm10의 이상치 제거
        # 데이터 "이상치" 처리 code -> 바로 앞의 값으로 대체하는 논리구조
        raw_df[raw_df['PM10'] > 1050.0].index
        outlier = raw_df[raw_df['PM10'] > 1050.0].index
        raw_df['PM10'].loc[outlier] = raw_df['PM10'].loc[outlier-1]

        # "서울 xxx" 와 같은 지역을 찾아내기 위해 '서울'이 포함된 지역만 검색
        raw_df = raw_df[raw_df['지역'].str.contains('서울')]

        # datetime형식으로 측정일시 type을 변경
        raw_df['측정일시'] = raw_df['측정일시'].astype('str')
        date_ls = list(raw_df['측정일시']) # 날짜 데이터를 list형태로 변경

        # 시간별로 date를 절사. :13까지가 시간단위!
        days_ls = []
        for date in date_ls:
            date = date[:8]
            days_ls.append(date)
        raw_df['측정일시'] = days_ls

        # 여기에, 다시 datetime 형식으로 만드는 코드 추가해야함!
        raw_df['측정일시'] = pd.to_datetime(raw_df['측정일시'], format='%Y%m%d')

        # 위에서 생성한 빈 데이터 프레임에 연결
        Merged_DF = pd.concat([Merged_DF, raw_df])
        Merged_DF = Merged_DF.reset_index(drop=True) # 인덱스 초기화

    #     print(len(raw_df)) # 분기별 데이터 수 확인
    #     print(raw_df['PM10'].max()) # 이상치 확인을 위함
    #     print(raw_df['PM25'].min()) # 이상치 확인을 위함 

    # 합쳐진 df을 다른 이름으로 빼내기
    print(Merged_DF.head()) # 데이터 확인 
    Merged_DF = Merged_DF.sort_values(by='지역') # 지역별로 오름차순 정렬
    Merged_DF = Merged_DF.reset_index(drop=True) # index 초기화
    Merged_DF = round(Merged_DF.groupby(['측정일시', '지역']).mean(),2) # 측정일시와 지역으로 groupby
    print(len(Merged_DF)) # 데이터 개수 확인 
    Merged_DF.to_csv(out_dir + 'total_{}.csv'.format(dir_name), encoding='cp949') # 데이터 쓰기

In [875]:
# DataPreprocessing('2017')

In [876]:
# DataPreprocessing('2018')

## 전처리가 끝난 2017, 2018 미세먼지 데이터 합치기
미세먼지외, 다른 데이터들과의 원활한 결합을 위하여 데이터 값들과 column의 이름을 통일화

모든 구 25개 X 730일(2년) = 18,250 행

In [877]:
data_dir = "C:\\Users\\seonmin\\NLP_Hause\\공공데이터 공모전\\test" # data가 들어있는 폴더 경로
# data_dir로 작업하는 폴더 경로 변경
working_dir = os.chdir(data_dir) 

_2017 = pd.read_csv('total_2017.csv',encoding='cp949') # 데이터 읽기
_2018 = pd.read_csv('total_2018.csv',encoding='cp949') # 데이터 읽기

_2017['지역'] = _2017['지역'].str.replace('서울 ', '') # 앞부분의 '서울 ' 제거. 구 이름만 남김
_2018['지역'] = _2018['지역'].str.replace('서울 ', '') # 앞부분의 '서울 ' 제거. 구 이름만 남김 

_2years = _2017.append(_2018) # 2017에 2018 데이터를 append
_2years = _2years.reset_index(drop=True) # 인덱스 초기화
_2years = _2years.rename(columns={'측정일시':'date'}) # 다른 데이터와의 결합을 위해 이름 변경
_2years.head()

,date,지역,CO,NO2,O3,PM10,PM25,SO2
0,2017-01-01,강남구,0.98,0.04,0.00,78.56,57.79,0.01
1,2017-01-01,강동구,0.88,0.04,0.00,81.25,54.21,0.00
2,2017-01-01,강북구,0.98,0.05,0.00,78.17,65.21,0.00
3,2017-01-01,강서구,0.90,0.06,0.00,84.08,NaN,0.01
4,2017-01-01,강서로,0.96,0.05,0.01,75.12,53.64,0.01


In [878]:
# 금천구의 경우 데이터가 중복
# 중구 청의 경우, 중구 청계천을 의미하는데, 중구라는 구가 있으므로 제외
# 강서로의 경우, 강서로를 의미하는데, 강서구라는 구가 있으므로 제외 
_2years = _2years[_2years['지역'] != '금천구']
_2years = _2years[_2years['지역'] != '중구 청']
_2years = _2years[_2years['지역'] != '강서로 ']

#### csv저장하기

In [879]:
# _2years.to_csv('2017X2018_서울미세먼지_합칠용.csv', encoding='cp949')

In [880]:
# mise = pd.read_csv('2017X2018_서울미세먼지_합칠용.csv', encoding='cp949', index_col=0)

#### datetime을 날짜형식 변환

In [881]:
mise = _2years

In [882]:
mise['date'] = pd.to_datetime(mise['date'])
type(mise['date'][0])
mise['지역'] = mise['지역'].str.replace(' ', '')
mise.head()

,date,지역,CO,NO2,O3,PM10,PM25,SO2
0,2017-01-01,강남구,0.98,0.04,0.00,78.56,57.79,0.01
1,2017-01-01,강동구,0.88,0.04,0.00,81.25,54.21,0.00
2,2017-01-01,강북구,0.98,0.05,0.00,78.17,65.21,0.00
3,2017-01-01,강서구,0.90,0.06,0.00,84.08,NaN,0.01
5,2017-01-01,관악구,1.26,0.06,0.01,74.96,55.75,0.01


## 5.2 공원df + 생활인구df + 미세먼지df 합치기

In [883]:
set(mise['지역']) == set(send['지역'])
flow_park_mise = send.merge(mise, on=['date','지역'] , how='left')

## 5.3 NO2 컬럼 삭제

분석에 큰 의미가 없다고 판단

In [884]:
flow_park_mise = flow_park_mise.drop('NO2', axis=1)

#### 생활인구+공원+미세먼지 csv 저장

In [885]:
# flow_park_mise.to_csv('flow_park_mise_cp949.csv', encoding='cp949', index=0)

# 공원df + 생활인구df + 미세먼지df 

# 6. 2017, 2018 일별 기상관측 
### 평균기온, 평균습도
2017과 2018숫자만 바꿔서 같은 작업진행함

## 주의: 2018년 강서구 자료 존재x


In [886]:
import pandas as pd

## 6.1 일별 기상관측 2017

In [887]:
nal_2017 = pd.read_csv("일별기상관측_2017.csv", encoding='utf-8')

In [888]:
nal_2017 = nal_2017.drop(['최저기온(℃)','최대기온(℃)','최저습도(%)','최고습도(%)','평균풍속(m/s)','최대풍속(m/s)','강수량(mm)'], axis =1)

In [889]:
nal_2017['지점명'].value_counts()
nal_2017 = nal_2017[nal_2017['지점명'] != '남산'].reset_index(drop=True)
nal_2017 = nal_2017.rename(columns = {'지점명':'지역'})
nal_2017['지역'] = nal_2017['지역'] + '구'
nal_2017.head()

,관측일자,지역,평균기온(℃),평균습도(%)
0,20170101,강남구,3.2,76.300003
1,20170101,강동구,3.6,78.099998
2,20170101,강북구,3.7,78.199997
3,20170101,강서구,4.6,77.800003
4,20170101,관악구,4.4,68.500000


In [890]:
nal_2017['지역'].value_counts()

강남구     359
동작구     359
용산구     359
성동구     359
중구구     359
광진구     359
구로구     359
동대문구    359
성북구     359
중랑구     359
금천구     359
양천구     359
강동구     359
마포구     359
송파구     359
은평구     357
노원구     357
서대문구    341
강북구     315
영등포구    278
관악구     278
강서구     264
도봉구     224
서초구     219
종로구     174
Name: 지역, dtype: int64

In [891]:
len(set(nal_2017['지역']) )

25

In [892]:
nal_2017.groupby('지역').mean()

,관측일자,평균기온(℃),평균습도(%)
지역,,,
강남구,2.017066e+07,13.655153,59.416434
강동구,2.017066e+07,13.898050,58.906685
강북구,2.017066e+07,11.947619,58.832063
강서구,2.017051e+07,15.610985,65.918182
관악구,2.017059e+07,12.393165,56.490647
광진구,2.017066e+07,14.161560,58.898329
구로구,2.017066e+07,13.821170,60.441783
금천구,2.017066e+07,14.020891,57.503900
노원구,2.017066e+07,14.031373,59.354902


## 6.2 일별 기상관측 2018

In [893]:
nal_2018 = pd.read_csv("일별기상관측_2018.csv", encoding='utf-8')

In [894]:
nal_2018 = nal_2018.drop(['최저기온(℃)','최대기온(℃)','최저습도(%)','최고습도(%)','평균풍속(m/s)','최대풍속(m/s)','강수량(mm)'], axis =1)

In [895]:
nal_2018['지점명'].value_counts()

동대문    343
은평     343
마포     343
광진     343
강북     343
강동     343
중구     343
용산     343
구로     343
동작     343
성북     343
강남     343
중랑     331
도봉     331
양천     320
성동     314
노원     307
서초     294
서대문    284
송파     278
남산     214
관악     194
영등포    184
금천     164
종로     126
Name: 지점명, dtype: int64

In [896]:
nal_2018 = nal_2018[nal_2018['지점명'] != '남산'].reset_index(drop=True)
nal_2018 = nal_2018.rename(columns = {'지점명':'지역'})
nal_2018['지역'] = nal_2018['지역'] + '구'
nal_2018.head()

,관측일자,지역,평균기온(℃),평균습도(%)
0,20180101,동대문구,-0.1,44.500000
1,20180101,성북구,-0.3,35.599998
2,20180101,도봉구,-1.6,50.000000
3,20180101,마포구,-1.0,52.000000
4,20180101,구로구,0.0,42.900002


#### 2017, 2018 구별 온도습도 csv 저장

In [897]:
# nal_2017.to_csv('구별온도습도2017.csv', encoding='utf-8', index=0)

In [898]:
# nal_2018.to_csv('구별온도습도2018.csv', encoding='utf-8', index=0)

## 6.3 2017과 2018 append해서 하나로 만듦

In [899]:
# seoul_2017 = pd.read_csv('구별온도습도2017.csv', encoding='utf-8')
# seoul_2018 = pd.read_csv('구별온도습도2018.csv', encoding='utf-8')

In [900]:
total_nal = nal_2017.append(nal_2018).reset_index(drop=True)

In [901]:
total_nal.head()

,관측일자,지역,평균기온(℃),평균습도(%)
0,20170101,강남구,3.2,76.300003
1,20170101,강동구,3.6,78.099998
2,20170101,강북구,3.7,78.199997
3,20170101,강서구,4.6,77.800003
4,20170101,관악구,4.4,68.500000


## 6.4 지역 이름이 '중구구' 라서 '중구'로 바꿔줌

#### part_1 = 중구

지역명 바꿔줌

In [902]:
part_1 = total_nal[total_nal['지역'] == '중구구']
part_1['지역'] = '중구'
part_1.head()

C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,관측일자,지역,평균기온(℃),평균습도(%)
23,20170101,중구,2.9,76.199997
48,20170102,중구,5.5,77.300003
73,20170103,중구,2.8,59.900002
98,20170104,중구,4.5,59.000000
123,20170105,중구,4.5,53.599998


#### part_2 중구가 아닌 모든 구

In [903]:
part_2 = total_nal[total_nal['지역'] != '중구구']

#### part_1 + part_2 다시 복원

In [904]:
total_nal = part_1.append(part_2).reset_index(drop=True)

In [905]:
total_nal = total_nal.rename(columns = {'관측일자':'date'})
total_nal['date'] = total_nal['date'].astype('str')
total_nal['date'] = pd.to_datetime(total_nal['date'])
total_nal.head()

,date,지역,평균기온(℃),평균습도(%)
0,2017-01-01,중구,2.9,76.199997
1,2017-01-02,중구,5.5,77.300003
2,2017-01-03,중구,2.8,59.900002
3,2017-01-04,중구,4.5,59.000000
4,2017-01-05,중구,4.5,53.599998


## 6.5 온도습도df + 기존 df

In [906]:
flow_park_mise['date'] = pd.to_datetime(flow_park_mise['date'])

In [907]:
flow_park_mise.head()

,date,지역,총생활인구수,공원면적,공원수,구 전체면적,미집행면적,예상총면적,1인당면적,공원당인구수,CO,O3,PM10,PM25,SO2
0,2017-01-01,강남구,668041.0,2146595.0,134,39703000.0,805207.0,2951802.0,3.0,4985.0,0.98,0.00,78.56,57.79,0.01
1,2017-01-01,강동구,505314.0,1934993.0,74,25236000.0,898745.0,2833738.0,4.0,6829.0,0.88,0.00,81.25,54.21,0.00
2,2017-01-01,강북구,332492.0,1567126.0,45,23819000.0,468932.0,2036058.0,5.0,7389.0,0.98,0.00,78.17,65.21,0.00
3,2017-01-01,강서구,563938.0,3209226.0,139,42077000.0,1472431.0,4681657.0,6.0,4057.0,0.90,0.00,84.08,NaN,0.01
4,2017-01-01,관악구,529131.0,442644.0,76,29969000.0,289624.0,732268.0,1.0,6962.0,1.26,0.01,74.96,55.75,0.01


In [908]:
flow_park_mise_nal = flow_park_mise.merge(total_nal, on=['date', '지역'], how='left')

In [909]:
flow_park_mise_nal.head()

,date,지역,총생활인구수,공원면적,공원수,구 전체면적,미집행면적,예상총면적,1인당면적,공원당인구수,CO,O3,PM10,PM25,SO2,평균기온(℃),평균습도(%)
0,2017-01-01,강남구,668041.0,2146595.0,134,39703000.0,805207.0,2951802.0,3.0,4985.0,0.98,0.00,78.56,57.79,0.01,3.2,76.300003
1,2017-01-01,강동구,505314.0,1934993.0,74,25236000.0,898745.0,2833738.0,4.0,6829.0,0.88,0.00,81.25,54.21,0.00,3.6,78.099998
2,2017-01-01,강북구,332492.0,1567126.0,45,23819000.0,468932.0,2036058.0,5.0,7389.0,0.98,0.00,78.17,65.21,0.00,3.7,78.199997
3,2017-01-01,강서구,563938.0,3209226.0,139,42077000.0,1472431.0,4681657.0,6.0,4057.0,0.90,0.00,84.08,NaN,0.01,4.6,77.800003
4,2017-01-01,관악구,529131.0,442644.0,76,29969000.0,289624.0,732268.0,1.0,6962.0,1.26,0.01,74.96,55.75,0.01,4.4,68.500000


# 7. 지하철 데이터 

# @@@ 1718년 평균? 1년 전체 평균??

## 7.1 지하철자료 불러오기

# @@@ 반장님 코드 받아오기

In [910]:
여기 지하철 코드추가

SyntaxError: invalid syntax (<ipython-input-910-c83f88a3d4ed>, line 1)

In [911]:
subway_raw = pd.read_csv('subway_gu_190817.csv', encoding='utf-8')

In [912]:
subway = subway_raw.copy()


In [913]:
subway = subway.rename(columns = {"구이름":'지역', '1718평균':'1718지하철평균'})
subway.head()

,지역,1718지하철평균
0,강남구,418599616
1,강동구,105849869
2,강북구,64751098
3,강서구,91431247
4,관악구,127196038


## 7.2 지하철df + 기존df

In [914]:
flow_park_subway =  flow_park_mise_nal.merge(subway, on='지역', how='left')

#### 지하철df + 기존df csv 저장하기

In [915]:
# flow_park_subway.to_csv('seoul_subway추가_cp949.csv', encoding='cp949', index = 0)

In [916]:
# flow_park_subway.to_csv('seoul_subway추가_utf8.csv', encoding='utf-8', index = 0)

# 8. 각 구청 위도 경도 추가

In [917]:
서초구청 37.483556, 127.032664
양천구청 37.516978, 126.866658
용산구청 37.532583, 126.990028
도봉구청 37.668711, 127.047087
성동구청 37.563448, 127.036956
광진구청 37.538526, 127.082384
은평구청 37.602774, 126.929144
관악구청 37.478117, 126.951506
영등포구청 37.526332, 126.896260
송파구청 37.514445, 127.105903
노원구청 37.654097, 127.056320
강남구청 37.517271, 127.047377
강서구청 37.550878, 126.849550
성북구청 37.589350, 127.016744
강북구청 37.639765, 127.025527
마포구청 37.566177, 126.901618
종로구청 37.573156, 126.979528
동작구청 37.512390, 126.939480
구로구청 37.495445, 126.887454
서대문구청 37.579156, 126.936785
중랑구청 37.606544, 127.092615
강동구청 37.530277, 127.123774
중구청 37.563734, 126.997546
동대문구청 37.574385, 127.039751
금천구청 37.456890, 126.895397



---------------수정 

강동구 37.552664, 127.144977
서초구 37.467901, 127.022618
동작구 37.501426, 126.957590
관악구 37.464856, 126.946504
노원구 37.651381, 127.084506

SyntaxError: invalid syntax (<ipython-input-917-5091f76413c9>, line 1)

In [918]:
gu_list = {'지역':['서초구','양천구','용산구','도봉구','성동구','광진구','은평구','관악구','영등포구','송파구','노원구','강남구','강서구','성북구','강북구','마포구','종로구','동작구','구로구','서대문구','중랑구','강동구','중구','동대문구','금천구'],
        '지역위도':['37.467901','37.516978','37.532583','37.668711','37.563448','37.538526','37.602774','37.464856','37.526332','37.514445','37.651381','37.517271','37.550878','37.589350','37.639765','37.566177','37.573156','37.501426','37.495445','37.579156','37.606544','37.552664','37.563734','37.574385','37.456890'],
          '지역경도':['127.022618','126.866658','126.990028','127.047087','127.036956','127.082384','126.929144','126.946504','126.896260','127.105903','127.084506','127.047377','126.849550','127.016744','127.025527','126.901618','126.979528','126.957590','126.887454','126.936785','127.092615','127.144977','126.997546','127.039751','126.895397']}

gu_location = pd.DataFrame(gu_list)
flow_park_subway = flow_park_subway.merge(gu_location, on='지역', how='left')

In [919]:
flow_park_subway.head()

,date,지역,총생활인구수,공원면적,공원수,구 전체면적,미집행면적,예상총면적,1인당면적,공원당인구수,CO,O3,PM10,PM25,SO2,평균기온(℃),평균습도(%),1718지하철평균,지역위도,지역경도
0,2017-01-01,강남구,668041.0,2146595.0,134,39703000.0,805207.0,2951802.0,3.0,4985.0,0.98,0.00,78.56,57.79,0.01,3.2,76.300003,418599616,37.517271,127.047377
1,2017-01-01,강동구,505314.0,1934993.0,74,25236000.0,898745.0,2833738.0,4.0,6829.0,0.88,0.00,81.25,54.21,0.00,3.6,78.099998,105849869,37.552664,127.144977
2,2017-01-01,강북구,332492.0,1567126.0,45,23819000.0,468932.0,2036058.0,5.0,7389.0,0.98,0.00,78.17,65.21,0.00,3.7,78.199997,64751098,37.639765,127.025527
3,2017-01-01,강서구,563938.0,3209226.0,139,42077000.0,1472431.0,4681657.0,6.0,4057.0,0.90,0.00,84.08,NaN,0.01,4.6,77.800003,91431247,37.550878,126.849550
4,2017-01-01,관악구,529131.0,442644.0,76,29969000.0,289624.0,732268.0,1.0,6962.0,1.26,0.01,74.96,55.75,0.01,4.4,68.500000,127196038,37.464856,126.946504


#### 위도경도 추가한 csv 저장하기

In [920]:
# flow_park_subway.to_csv('seoul_subway추가_cp949.csv', encoding='cp949', index = 0)

In [921]:
# flow_park_subway.to_csv('seoul_subway추가_utf8.csv', encoding='utf-8', index = 0)

# 각 지역별 데이터로 변환

In [922]:
지도에 그림 그리기위해 평균으로 바꾸고 행을 줄인건데(25행으로) 이게 맞는 일인지 모르겠음.
그런데 우리는 여기서 행을 줄인 df로 이후 분석작업을 진행했기 때문에 
이것에 대한 근거가 명확하게 있어야함..

SyntaxError: invalid syntax (<ipython-input-922-2bc985c7f3b8>, line 1)

In [923]:
test_group = flow_park_subway.groupby('지역')
test_df = test_group.mean()
test_df = test_df.reset_index()

#### 시각화를 위해 위도,경도 추가

In [924]:
test_df = test_df.merge(gu_location, on='지역',how='left')

In [925]:
test_df.head()

,지역,총생활인구수,공원면적,공원수,구 전체면적,미집행면적,예상총면적,1인당면적,공원당인구수,CO,O3,PM10,PM25,SO2,평균기온(℃),평균습도(%),1718지하철평균,지역위도,지역경도
0,강남구,821452.478082,2146595.0,134,39703000.0,805207.0,2951802.0,2.919178,6130.234247,0.580671,0.018123,44.865384,24.330179,0.004178,13.447293,59.213675,418599616,37.517271,127.047377
1,강동구,485604.661644,1934993.0,74,25236000.0,898745.0,2833738.0,4.005479,6562.195890,0.525370,0.019644,46.526164,23.535507,0.001753,13.819231,54.903276,105849869,37.552664,127.144977
2,강북구,308546.143836,1567126.0,45,23819000.0,468932.0,2036058.0,5.004110,6856.573973,0.459342,0.030247,36.125589,20.430740,0.000863,12.775532,53.084954,64751098,37.639765,127.025527
3,강서구,547291.173973,3209226.0,139,42077000.0,1472431.0,4681657.0,6.005479,3937.347945,0.529644,0.015438,57.170260,25.953832,0.008616,15.610985,65.918182,91431247,37.550878,126.849550
4,관악구,502414.273973,442644.0,76,29969000.0,289624.0,732268.0,1.000000,6610.708219,0.464834,0.023657,43.192971,26.632455,0.004587,12.658475,55.870551,127196038,37.464856,126.946504


# 9. '도시농업공원' 이슈 발생

### 도시농업공원 노원구 & 양천구 면적 분석에서 제외 결정

## 9.1 인덱스를 '지역'으로 변경

In [926]:
test_df = test_df.set_index('지역')

## 9.2 노원구의 도시농업공원 2곳의 면적 제외

### 이름 : 수락산 공원, 불암산 공원

In [927]:
test_df['공원면적']['노원구'] = test_df['공원면적']['노원구'] - (6692795 + 5334267)

C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## 9.3 양천구의 도시농업공원 1곳 면적 제외

### 이름 : 온수도시자연공원

In [928]:
test_df['공원면적']['양천구'] = test_df['공원면적']['양천구'] - 1003471

C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [929]:
test_df.head()

,총생활인구수,공원면적,공원수,구 전체면적,미집행면적,예상총면적,1인당면적,공원당인구수,CO,O3,PM10,PM25,SO2,평균기온(℃),평균습도(%),1718지하철평균,지역위도,지역경도
지역,,,,,,,,,,,,,,,,,,
강남구,821452.478082,2146595.0,134,39703000.0,805207.0,2951802.0,2.919178,6130.234247,0.580671,0.018123,44.865384,24.330179,0.004178,13.447293,59.213675,418599616,37.517271,127.047377
강동구,485604.661644,1934993.0,74,25236000.0,898745.0,2833738.0,4.005479,6562.195890,0.525370,0.019644,46.526164,23.535507,0.001753,13.819231,54.903276,105849869,37.552664,127.144977
강북구,308546.143836,1567126.0,45,23819000.0,468932.0,2036058.0,5.004110,6856.573973,0.459342,0.030247,36.125589,20.430740,0.000863,12.775532,53.084954,64751098,37.639765,127.025527
강서구,547291.173973,3209226.0,139,42077000.0,1472431.0,4681657.0,6.005479,3937.347945,0.529644,0.015438,57.170260,25.953832,0.008616,15.610985,65.918182,91431247,37.550878,126.849550
관악구,502414.273973,442644.0,76,29969000.0,289624.0,732268.0,1.000000,6610.708219,0.464834,0.023657,43.192971,26.632455,0.004587,12.658475,55.870551,127196038,37.464856,126.946504


# 10.  취약계층 데이터

## @@@@@@ 취약계층 csv ???

## 마지막날 취약계층

In [930]:
weak = pd.read_csv('취약계층비율(0_14,65세이상).csv', encoding='utf-8')
weak = weak.set_index('지역')

In [931]:
weak

,취약계층비율
지역,
종로구,0.285
중구,0.230
용산구,0.310
성동구,0.337
광진구,0.318
동대문구,0.377
중랑구,0.416
성북구,0.404
강북구,0.465


## 10.1 취약계층 불러오기

In [932]:
# weak = pd.read_csv('취약계층비율.csv', encoding='utf-8')
# weak = weak.set_index('지역')

In [933]:
# weak.head()

## 10.2 기존df + 취약계층비율df

In [934]:
test_df = test_df.merge(weak, on='지역', how='left')

## 10.3 불투수면적 & 구 전체면적df + 기존df

### 10.3.1 서울시 지역df 만들기

In [935]:
gu_str = """강남구 강동구 강북구 강서구 관악구 광진구 구로구 금천구 노원구 도봉구 동대문구 동작구 마포구 서대문구 서초구 성동구 성북구 송파구 양천구 영등포구 용산구 은평구 종로구 중구 중랑구"""
gu_list = gu_str.split()
gu_list_df = pd.DataFrame(gu_list).rename(columns ={0:'지역'}).reset_index()

In [936]:
gu_list_df.head()

,index,지역
0,0,강남구
1,1,강동구
2,2,강북구
3,3,강서구
4,4,관악구


### 10.3.2 구 전체면적 df만들기

In [937]:
gu_map = """39703000
25236000
23819000
42077000
29969000
17874000
20096000
12946000
35755000
20777000
14588000
16457000
23479000
17813000
46956000
16422000
24877000
33109000
17346000
24639000
21968000
31333000
23128000
9980000
18314000"""




In [938]:
# gu_map_list = gu_map.split('\n')

In [939]:
# gu_map_df = pd.DataFrame(gu_map_list).rename(columns = {0:'구 전체면적'}).reset_index()

In [940]:
# gu_map_df.head()

### 10.3.3 불투수면적 df만들기

In [941]:
bul_str = """21460000
12054000
8615000
14959000
11763000
10188000
11839000
7785000
13089000
8362000
10843000
10171000
12757000
9687000
16667000
10619000
13503000
17715000
11711000
14660000
9663000
12224000
9711000
7525000
10471000
"""

In [942]:
bul_list = bul_str.split('\n')
bul_list_df = pd.DataFrame(bul_list).rename(columns = {0:'불투수면적'}).reset_index()

### 10.3.4 구전체면적df + 불투수면적df

In [943]:
# imsi = gu_list_df.merge(gu_map_df, on ='index').merge(bul_list_df, on='index').set_index('지역')

In [944]:
imsi = gu_list_df.merge(bul_list_df, on='index').set_index('지역')

In [945]:
imsi.head()

,index,불투수면적
지역,,
강남구,0,21460000
강동구,1,12054000
강북구,2,8615000
강서구,3,14959000
관악구,4,11763000


In [946]:
imsi = imsi.drop('index', axis = 1)

## *1인당 면적 수정

In [947]:
test_df['1인당면적'] = 0

In [948]:
test_df['1인당면적'] = test_df['공원면적'] / test_df['총생활인구수']

# 최종 dataframe

In [949]:
test_df = test_df.merge(imsi, on='지역', how='left')
df_final = test_df

In [950]:
df_final

,총생활인구수,공원면적,공원수,구 전체면적,미집행면적,예상총면적,1인당면적,공원당인구수,CO,O3,PM10,PM25,SO2,평균기온(℃),평균습도(%),1718지하철평균,지역위도,지역경도,취약계층비율,불투수면적
지역,,,,,,,,,,,,,,,,,,,,
강남구,821452.478082,2146595.0,134,39703000.0,805207.0,2951802.0,2.613170,6130.234247,0.580671,0.018123,44.865384,24.330179,0.004178,13.447293,59.213675,418599616,37.517271,127.047377,0.265,21460000
강동구,485604.661644,1934993.0,74,25236000.0,898745.0,2833738.0,3.984708,6562.195890,0.525370,0.019644,46.526164,23.535507,0.001753,13.819231,54.903276,105849869,37.552664,127.144977,0.401,12054000
강북구,308546.143836,1567126.0,45,23819000.0,468932.0,2036058.0,5.079065,6856.573973,0.459342,0.030247,36.125589,20.430740,0.000863,12.775532,53.084954,64751098,37.639765,127.025527,0.465,8615000
강서구,547291.173973,3209226.0,139,42077000.0,1472431.0,4681657.0,5.863837,3937.347945,0.529644,0.015438,57.170260,25.953832,0.008616,15.610985,65.918182,91431247,37.550878,126.849550,0.382,14959000
관악구,502414.273973,442644.0,76,29969000.0,289624.0,732268.0,0.881034,6610.708219,0.464834,0.023657,43.192971,26.632455,0.004587,12.658475,55.870551,127196038,37.464856,126.946504,0.312,11763000
광진구,385677.543836,43788.0,38,17874000.0,12634.0,56422.0,0.113535,10149.395890,0.575036,0.024068,40.480979,22.643393,0.003208,13.962393,58.734188,162927094,37.538526,127.082384,0.318,10188000
구로구,402736.628767,2705529.0,47,20096000.0,313974.0,3019503.0,6.717862,8568.847945,0.437806,0.024486,43.919861,23.835542,0.006514,13.617379,59.541595,215541780,37.495445,126.887454,0.358,11839000
금천구,225497.327397,147893.0,51,12946000.0,418885.0,566778.0,0.655853,4421.517808,0.492767,0.024438,40.139192,24.308370,0.002082,12.535182,55.291396,66568118,37.456890,126.895397,0.302,7785000
노원구,534145.841096,2430715.0,118,35755000.0,852658.0,15310435.0,4.550658,4526.656164,0.554397,0.020712,43.675466,23.785041,0.002740,12.962349,58.690211,164274192,37.651381,127.084506,0.414,13089000


In [951]:
# df_final.to_csv('공공마지막날최종.csv', encoding='euc_kr')

## @@@ 마지막날

In [952]:
# df_final = pd.read_csv('최종시트_1인당면적 수정.csv', encoding = 'utf-8').set_index('지역')

In [953]:
# df_final

# 11. final rank 4가지 기준에 부합하는 지역 찾기

## 11.1 미세먼지 높은구

In [954]:
list_1 = list(df_final['PM25'].sort_values(ascending = [False]).index)

In [955]:
df_1 = pd.DataFrame(list_1)
df_1 = df_1.rename(columns = {0 : '지역'})
df_1['미세수치순위'] = 0
for i in range(25):
    df_1['미세수치순위'][i] = i+1
    
df_1['미세수치점수'] = 0
for i in range(25):
    df_1['미세수치점수'][i] = 25-i
    
df_1

C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,지역,미세수치순위,미세수치점수
0,관악구,1,25
1,마포구,2,24
2,강서구,3,23
3,양천구,4,22
4,영등포구,5,21
5,성북구,6,20
6,용산구,7,19
7,성동구,8,18
8,은평구,9,17
9,동작구,10,16


## 11.2 공원면적비율 적은 수

## 11.2.1 파생변수 - 공원면적비율 생성

In [956]:
df_final['구전체면적'] = df_final['구전체면적'].astype('float')
df_final['공원면적비율'] = df_final['공원면적'] / df_final['구전체면적']

KeyError: '구전체면적'

In [957]:
list_2 = list(df_final['공원면적비율'].sort_values(ascending = [True]).index)

KeyError: '공원면적비율'

In [958]:
df_2 = pd.DataFrame(list_2)
df_2 = df_2.rename(columns = {0 : '지역'})
df_2['공원비율적은순위'] = 0
for i in range(25):
    df_2['공원비율적은순위'][i] = i+1
df_2['공원비율적은점수'] = 0
for i in range(25):
    df_2['공원비율적은점수'][i] = 25-i
    
df_2

C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,지역,공원비율적은순위,공원비율적은점수
0,광진구,1,25
1,금천구,2,24
2,영등포구,3,23
3,관악구,4,22
4,용산구,5,21
5,중구,6,20
6,서초구,7,19
7,성동구,8,18
8,마포구,9,17
9,도봉구,10,16


## 11.3 흡수율 적은 수 (분석 제외)

# @@@미세먼지 흡수량 계산법??

In [959]:
# 옛버전
# df_final['미세먼지흡수량(m2)'].sort_values(ascending = [True])[:10]

In [960]:
# list_3 = list(df_final['미세먼지흡수량(m2)'].sort_values(ascending = [True]).index)

In [961]:
# 옛버전
# 흡수율 적은 수 정렬

# list_3 = ['광진구' , '금천구' , '중구' , '영등포구' , '관악구' , '용산구' , '성동구' , '동대문구' , '도봉구' ,'마포구' ]

In [962]:
# df_3 = pd.DataFrame(list_3)
# df_3 = df_3.rename(columns = {0 : '지역'})
# df_3['흡수적은순위'] = 0
# for i in range(25):
#     df_3['흡수적은순위'][i] = i+1
# df_3['흡수적은점수'] = 0
# for i in range(25):
#     df_3['흡수적은점수'][i] = 25-i
    
# df_3

## 11.4.1 취약계층 비율(분석 제외)

In [963]:
# list_4 = list(weak['취약계층비율'].sort_values(ascending = [False]).index)

In [964]:
# df_4 = pd.DataFrame(list_4)
# df_4 = df_4.rename(columns = {0 : '지역'})
# df_4['취약높은순위'] = 0
# for i in range(25):
#     df_4['취약높은순위'][i] = i+1
# df_4['취약높은점수'] = 0
# for i in range(25):
#     df_4['취약높은점수'][i] = 25-i
    
# df_4

## 11.4.2 취약계층 수

In [965]:
df_final['취약계층수'] = df_final['총생활인구수'] * df_final['취약계층비율']

In [966]:
list_4 = list(df_final['취약계층수'].sort_values(ascending = [False]).index)

In [967]:
df_4 = pd.DataFrame(list_4)
df_4 = df_4.rename(columns = {0 : '지역'})
df_4['취약높은순위'] = 0
for i in range(25):
    df_4['취약높은순위'][i] = i+1
df_4['취약높은점수'] = 0
for i in range(25):
    df_4['취약높은점수'][i] = 25-i
    
df_4

C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,지역,취약높은순위,취약높은점수
0,송파구,1,25
1,노원구,2,24
2,강남구,3,23
3,강서구,4,22
4,은평구,5,21
5,강동구,6,20
6,서초구,7,19
7,성북구,8,18
8,양천구,9,17
9,관악구,10,16


## 11. 5 공원1m2당 인구수 (분석 제외)

In [968]:
# df_final['1m2인구수'] = df_final['총생활인구수'] / df_final['공원면적']

In [969]:
# list_5 = list(df_final['1m2인구수'].sort_values(ascending = [False]).index)

In [970]:
# df_5 = pd.DataFrame(list_5)
# df_5 = df_5.rename(columns = {0 : '지역'})
# df_5['1m2인구수'] = 0
# for i in range(25):
#     df_5['1m2인구수'][i] = i+1
# df_5['1m2인구점수'] = 0
# for i in range(25):
#     df_5['1m2인구점수'][i] = 25-i
    
# df_5

## 11.6 1인당 공원면적

In [971]:
df_final['1인당공원면적'] =  df_final['공원면적'] / df_final['총생활인구수']

In [972]:
list_6 = list(df_final['1인당공원면적'].sort_values(ascending = [True]).index)

In [973]:
df_6 = pd.DataFrame(list_6)
df_6 = df_6.rename(columns = {0 : '지역'})
df_6['1인당공원면적'] = 0
for i in range(25):
    df_6['1인당공원면적'][i] = i+1
df_6['1인당공원점수'] = 0
for i in range(25):
    df_6['1인당공원점수'][i] = 25-i
    
df_6

C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\SEONMIN\Anaconda3\envs\study_2\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,지역,1인당공원면적,1인당공원점수
0,광진구,1,25
1,영등포구,2,24
2,금천구,3,23
3,중구,4,22
4,관악구,5,21
5,용산구,6,20
6,성동구,7,19
7,마포구,8,18
8,동대문구,9,17
9,강남구,10,16


## 11.7  4개 df 합치기

In [974]:
rank_test = df_1.merge(df_2, on='지역',how='outer').merge(df_4, on='지역',how='outer').merge(df_6, on='지역',how='outer')

In [975]:
# 1m2인구수 버전
# rank_test = df_1.merge(df_2, on='지역',how='outer').merge(df_4, on='지역',how='outer').merge(df_5, on='지역',how='outer')

In [976]:
rank_test

,지역,미세수치순위,미세수치점수,공원비율적은순위,공원비율적은점수,취약높은순위,취약높은점수,1인당공원면적,1인당공원점수
0,관악구,1,25,4,22,10,16,5,21
1,마포구,2,24,9,17,18,8,8,18
2,강서구,3,23,19,7,4,22,21,5
3,양천구,4,22,17,9,9,17,12,14
4,영등포구,5,21,3,23,12,14,2,24
5,성북구,6,20,12,14,8,18,13,13
6,용산구,7,19,5,21,23,3,6,20
7,성동구,8,18,8,18,21,5,7,19
8,은평구,9,17,25,1,5,21,25,1
9,동작구,10,16,23,3,13,13,22,4


In [977]:
rank_test = rank_test.fillna(0)

In [978]:
rank_test['총합'] = rank_test['미세수치점수'] + rank_test['공원비율적은점수'] + rank_test['1인당공원점수'] +rank_test['취약높은점수']

In [979]:
rank_test = rank_test.sort_values(['총합'], ascending=[False]).reset_index(drop=True)


## 11.8  최종 순위

In [980]:
rank_test['미세수치순위'] = rank_test['미세수치순위'].astype('int')
rank_test['미세수치점수'] = rank_test['미세수치점수'].astype('int')
rank_test['공원비율적은순위'] = rank_test['공원비율적은순위'].astype('int')
rank_test['공원비율적은점수'] = rank_test['공원비율적은점수'].astype('int')
rank_test['1인당공원면적'] = rank_test['1인당공원면적'].astype('int')
rank_test['1인당공원점수'] = rank_test['1인당공원점수'].astype('int')
rank_test['취약높은순위'] = rank_test['취약높은순위'].astype('int')
rank_test['취약높은점수'] = rank_test['취약높은점수'].astype('int')


In [981]:
# 1m2인구수 버전
rank_test

,지역,미세수치순위,미세수치점수,공원비율적은순위,공원비율적은점수,취약높은순위,취약높은점수,1인당공원면적,1인당공원점수,총합
0,관악구,1,25,4,22,10,16,5,21,84
1,영등포구,5,21,3,23,12,14,2,24,82
2,강남구,11,15,11,15,3,23,10,16,69
3,마포구,2,24,9,17,18,8,8,18,67
4,성북구,6,20,12,14,8,18,13,13,65
5,용산구,7,19,5,21,23,3,6,20,63
6,서초구,17,9,7,19,7,19,11,15,62
7,양천구,4,22,17,9,9,17,12,14,62
8,금천구,12,14,2,24,25,1,3,23,62
9,성동구,8,18,8,18,21,5,7,19,60


In [982]:
# 지역 이름으로 정렬
rank_test.sort_values(['지역'], ascending=[True]).reset_index(drop=True)

,지역,미세수치순위,미세수치점수,공원비율적은순위,공원비율적은점수,취약높은순위,취약높은점수,1인당공원면적,1인당공원점수,총합
0,강남구,11,15,11,15,3,23,10,16,69
1,강동구,18,8,20,6,6,20,16,10,44
2,강북구,25,1,15,11,15,11,20,6,29
3,강서구,3,23,19,7,4,22,21,5,57
4,관악구,1,25,4,22,10,16,5,21,84
5,광진구,23,3,1,25,20,6,1,25,59
6,구로구,15,11,22,4,14,12,23,3,30
7,금천구,12,14,2,24,25,1,3,23,62
8,노원구,16,10,16,10,2,24,18,8,52
9,도봉구,19,7,10,16,17,9,14,12,44


#### 최종순위 csv 저장

In [983]:
# rank_test.to_csv('rank_last.csv', encoding="euc-kr", index=0)

In [512]:
# df_final.to_csv('[공공]마지막날_최종데이터.csv',encoding="euc-kr") 